In [25]:
!python --version

Python 3.11.5


In [26]:
!pip install selenium

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [27]:
!pip install beautifulsoup4

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [28]:
!pip install webdriver-manager

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [30]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import time

In [31]:
import re
import pandas as pd

In [32]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [33]:
def web_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    options.add_argument('--window-size=1920, 1200')
    options.add_argument('--lang=pt-BR')
    # options.add_argument('--lang=en-US')
    
    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()),
        options=options
    )
    
    return driver

# web_driver()

def open_page(url):
    driver = web_driver()
    driver.get(url)
    soup = get_page_source(driver)
    return driver, soup
    
def get_page_source(driver):
    res = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(res, 'html.parser')
    return soup

In [34]:
driver, soup = open_page('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc')

button = WebDriverWait(driver, 10) \
            .until(EC.element_to_be_clickable((By.CLASS_NAME, 'ipc-see-more__button')))

ActionChains(driver).move_to_element(button).click().perform()

time.sleep(5)

soup = get_page_source(driver)

# print(soup.prettify().splitlines()[:10])
ul_movies = soup.find('ul', attrs={'class': 'ipc-metadata-list'})
li_movies = ul_movies.find_all('li', attrs={'class': 'ipc-metadata-list-summary-item'})

movies = []

for li_movie in li_movies:
    link = li_movie.find('div', attrs={'class': 'ipc-title'}).find('a')['href']
    id = link.split('/')[2]
    title = li_movie.find('div', attrs={'class': 'ipc-title'}).find('a').text
    title = re.sub(r'\d+\.\s', '', title)
    
    metadata = li_movie.find('div', attrs={'class': 'dli-title-metadata'})
    
    #year = metadata.find_all('span')[0].text
    year = metadata.find('span', string=re.compile(r'\b\d{4}\b')).text
    
    #duration = metadata.find_all('span')[1].text
    duration = metadata.find('span', string=re.compile(r'\b\d{1,2}h(\s\d{1,2}m)?\b')).text
    
    try:
        certificate = metadata.find_all('span')[2].text
    except:
        certificate = '-'
        
    rating = li_movie.find('span', attrs={'class': 'ipc-rating-star--rating'}).text
    rating = rating.replace(',','.').strip() #trim
    
    vote_count = li_movie.find('span', attrs={'class': 'ipc-rating-star--voteCount'}).text
    vote_count = re.sub(r'[()]', '', vote_count).replace(',','.').strip() #Strip -> Retirar os espaços em branco
    vote_count = vote_count.replace(u'\xa0', u' ')
    
    metascore = li_movie.find('span', attrs={'class': 'metacritic-score-box'})
    metascore = metascore.text if metascore else '-'
    
    info = li_movie.find('div', attrs={'class': 'ipc-html-content-inner-div'}).text
    
    #print(id, title, link, year, duration, certificate, rating, vote_count, metascore, info)
    
    movies.append({
        'id': id,
        'title': title,
        'link': link,
        'year': year,
        'duration': duration,
        'certificate': certificate,
        'rating': rating,
        'vote_count': vote_count,
        'metascore': metascore,
        'info': info
    })
    
driver.quit()

In [35]:
print(len(movies))
movies[:3]

100


[{'id': 'tt0111161',
  'title': 'Um Sonho de Liberdade',
  'link': '/title/tt0111161/?ref_=sr_t_1',
  'year': '1994',
  'duration': '2h 22m',
  'certificate': '16',
  'rating': '9.3',
  'vote_count': '2.9 mi',
  'metascore': '82',
  'info': 'Dois homens presos se reúnem ao longo de vários anos, encontrando consolo e eventual redenção através de atos de decência comum.'},
 {'id': 'tt0068646',
  'title': 'O Poderoso Chefão',
  'link': '/title/tt0068646/?ref_=sr_t_2',
  'year': '1972',
  'duration': '2h 55m',
  'certificate': '14',
  'rating': '9.2',
  'vote_count': '2 mi',
  'metascore': '100',
  'info': 'O patriarca idoso de uma dinastia do crime organizado transfere o controle de seu império clandestino para seu filho relutante.'},
 {'id': 'tt0468569',
  'title': 'Batman: O Cavaleiro das Trevas',
  'link': '/title/tt0468569/?ref_=sr_t_3',
  'year': '2008',
  'duration': '2h 32m',
  'certificate': '12',
  'rating': '9.0',
  'vote_count': '2.9 mi',
  'metascore': '84',
  'info': 'Agora c

In [36]:
df = pd.DataFrame.from_dict(movies)

for column in ['year', 'rating', 'metascore']:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df.dtypes

id              object
title           object
link            object
year             int64
duration        object
certificate     object
rating         float64
vote_count      object
metascore      float64
info            object
dtype: object

In [37]:
# top 10 maior quantidade de filmes entre o top 100
df.value_counts(subset=['year']) \
    .to_frame().reset_index() \
    .sort_values(by=['count', 'year'], ascending=[False, False], ignore_index=True).head(10)

,year,count
0,1994,5
1,1999,4
2,1995,4
3,2019,3
4,2018,3
5,2012,3
6,2006,3
7,2002,3
8,2000,3
9,1997,3


In [38]:
df.head(2)

,id,title,link,year,duration,certificate,rating,vote_count,metascore,info
0,tt0111161,Um Sonho de Liberdade,/title/tt0111161/?ref_=sr_t_1,1994,2h 22m,16,9.3,2.9 mi,82.0,"Dois homens presos se reúnem ao longo de vários anos, encontrando consolo e eventual redenção através de atos de decência comum."
1,tt0068646,O Poderoso Chefão,/title/tt0068646/?ref_=sr_t_2,1972,2h 55m,14,9.2,2 mi,100.0,O patriarca idoso de uma dinastia do crime organizado transfere o controle de seu império clandestino para seu filho relutante.


In [39]:
def to_minutes(duration):
    parts = duration.split()
    hours = minutes = 0
    for part in parts:
        if 'h' in part:
            hours = int(part.replace('h', ''))
        elif 'm' in part:
            minutes = int(part.replace('m', ''))
    return hours * 60 + minutes

print(to_minutes('2h 22m'))
print(to_minutes('2h'))

142
120


In [40]:
df['duration-min'] = df['duration'].apply(to_minutes)
df.head(2)

,id,title,link,year,duration,certificate,rating,vote_count,metascore,info,duration-min
0,tt0111161,Um Sonho de Liberdade,/title/tt0111161/?ref_=sr_t_1,1994,2h 22m,16,9.3,2.9 mi,82.0,"Dois homens presos se reúnem ao longo de vários anos, encontrando consolo e eventual redenção através de atos de decência comum.",142
1,tt0068646,O Poderoso Chefão,/title/tt0068646/?ref_=sr_t_2,1972,2h 55m,14,9.2,2 mi,100.0,O patriarca idoso de uma dinastia do crime organizado transfere o controle de seu império clandestino para seu filho relutante.,175


In [41]:
df.sort_values(by=['duration-min', 'title'], ascending=[True, True], ignore_index=True).head(3)

,id,title,link,year,duration,certificate,rating,vote_count,metascore,info,duration-min
0,tt0114709,Toy Story,/title/tt0114709/?ref_=sr_t_88,1995,1h 21m,Livre,8.3,1.1 mi,96.0,Um brinquedo do cauboi se sente profundamente ameaçado e celoso quando um brinquedo patrulheiro espacial passa a ter mais atençao do garoto.,81
1,tt0021749,Luzes da Cidade,/title/tt0021749/?ref_=sr_t_56,1931,1h 27m,Livre,8.5,198 mil,99.0,"Com a ajuda de um homem rico, bêbado e imprevisível, um vagabundo ingênuo que se apaixonou por uma florista cega tenta conseguir dinheiro para fornecer ajuda médica.",87
2,tt0027977,Tempos Modernos,/title/tt0027977/?ref_=sr_t_55,1936,1h 27m,Livre,8.5,263 mil,96.0,Um vagabundo luta para viver na sociedade industrial moderna com a ajuda de uma jovem.,87


In [42]:
df.sort_values(by=['duration-min', 'title'], ascending=[True, True], ignore_index=True).tail(3)

,id,title,link,year,duration,certificate,rating,vote_count,metascore,info,duration-min
97,tt0047478,Os Sete Samurais,/title/tt0047478/?ref_=sr_t_31,1954,3h 27m,10,8.6,372 mil,98.0,Uma aldeia pobre sob a ameaça de bandidos recruta sete samurais para ajudá-los a se defenderem.,207
98,tt0056172,Lawrence da Arábia,/title/tt0056172/?ref_=sr_t_94,1962,3h 38m,14,8.3,319 mil,100.0,"A história de T. E. Lawrence, o oficial inglês que conseguiu unir e liderar várias tribos árabes, durante a Primeira Guerra Mundial para lutar contra os turcos.",218
99,tt0087843,Era uma Vez na América,/title/tt0087843/?ref_=sr_t_93,1984,3h 49m,18,8.3,383 mil,75.0,"Dois amigos leais crescem juntos cometendo pequenos crimes nas ruas do Lower East Side, em Manhattan, mas essa parceria com o tempo se desfaz em morte e mistério.",229


In [43]:
movies[0]

{'id': 'tt0111161',
 'title': 'Um Sonho de Liberdade',
 'link': '/title/tt0111161/?ref_=sr_t_1',
 'year': '1994',
 'duration': '2h 22m',
 'certificate': '16',
 'rating': '9.3',
 'vote_count': '2.9 mi',
 'metascore': '82',
 'info': 'Dois homens presos se reúnem ao longo de vários anos, encontrando consolo e eventual redenção através de atos de decência comum.'}

In [44]:
def find_non_none(tag1, tag2):
    if tag1:
        return tag1
    elif tag2:
        return tag2
    return None

In [57]:
for movie in movies[:1]:
    link = f'https://www.imdb.com{movie["link"]}'
    print(movie['id'], movie['title'], link)
    
    driver, soup = open_page(link)
    
    interests = soup. find('div', attrs={'class': 'ipc-chip-list__scroller'}) \
        .find_all('a', attrs={'class': 'ipc-chip'})
    interests = [interest.text for interest in interests]
    #print(interests)
    movie['interests'] = interests
    
    # ---------
    
    movie_metadata = soup.find('ul', attrs={'class': 'ipc-metadata-list'})
    
    directors = find_non_none(
                            movie_metadata.find('span', string='Direção'),
                            movie_metadata.find('a', string='Direção')
                        ) \
                        .find_parent().find_all('li', attrs={'class': 'ipc-inline-list__item'})
    directors = [directors.text for directors in directors]
    #print(directors)
    movie['directors'] = directors
    
    writers = find_non_none(
                            movie_metadata.find('span', string=re.compile(r'Roteirista(s)?')),
                            movie_metadata.find('a', string=re.compile(r'Roteirista(s)?'))
                        ) \
                        .find_parent().find_all('li', attrs={'class': 'ipc-inline-list__item'})
    writers = [writers.text for writers in writers]
    #print(writers)
    movie['writers'] = writers
    
    stars = find_non_none(
                            movie_metadata.find('span', string=re.compile(r'Artistas')),
                            movie_metadata.find('a', string=re.compile(r'Artistas'))
                        ) \
                        .find_parent().find_all('li', attrs={'class': 'ipc-inline-list__item'})
    stars = [star.text for star in stars]
    #print(writers)
    movie['stars'] = stars
    
    # ---
    
    cast_actor_chars = []
    
    cast_cards = soup.find_all('div', attrs={'data-testid': 'title-cast-item'})
    cast_chars = []
    for cast_card in cast_cards:
        cast_actor = cast_card.find('a', attrs={'data-testid': 'title-cast-item__actor'}).text
        #print(cast_actor)
        if cast_card.find('div', attrs={'class': 'title-cast-item__characters-list'}):
            cast_chars = cast_card \
                            .find('div', attrs={'class': 'title-cast-item__characters-list'}) \
                            .find('li', attrs={'class': 'ipc-inline-list__item'})
            cast_chars = [cast_char.text for cast_char in cast_chars]
            #print(cast_chars)
        cast_actor_chars.append((cast_actor, cast_chars))
    movie['cast_actor_chars'] = cast_actor_chars
    
    # ---
    
    details_section = soup.find('section', attrs={'data-testid': 'Details'})
    
    release_date = details_section.find('a', string='Data de lançamento') \
                                  .find_parent().find_all()[1].text
    
    country_origin = details_section.find('span', string=re.compile(r'País(es)? de origem')) \
                                    .find_parent().find_all('li')
    
    country_origin = [country.text for country in country_origin]
    
    language = details_section.find('span', string=re.compile(r'Idioma(s)?')) \
                                    .find_parent().find_all('li')
    
    language = [lang.text for lang in language]
    
    movie['details'] = {
        'release_date': release_date,
        'country_origin': country_origin,
        'language': language
    }
    
    driver.quit()

tt0111161 Um Sonho de Liberdade https://www.imdb.com/title/tt0111161/?ref_=sr_t_1


In [58]:
movies[0]

{'id': 'tt0111161',
 'title': 'Um Sonho de Liberdade',
 'link': '/title/tt0111161/?ref_=sr_t_1',
 'year': '1994',
 'duration': '2h 22m',
 'certificate': '16',
 'rating': '9.3',
 'vote_count': '2.9 mi',
 'metascore': '82',
 'info': 'Dois homens presos se reúnem ao longo de vários anos, encontrando consolo e eventual redenção através de atos de decência comum.',
 'interests': ['Epic', 'Period Drama', 'Prison Drama', 'Tragedy', 'Drama'],
 'directors': ['Frank Darabont'],
 'writers': ['Stephen King', 'Frank Darabont'],
 'stars': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'],
 'cast_actor_chars': [('Tim Robbins', ['Andy Dufresne']),
  ('Morgan Freeman', ["Ellis Boyd 'Red' Redding"]),
  ('Bob Gunton', ['Warden Norton']),
  ('William Sadler', ['Heywood']),
  ('Clancy Brown', ['Captain Hadley']),
  ('Gil Bellows', ['Tommy']),
  ('Mark Rolston', ['Bogs Diamond']),
  ('James Whitmore', ['Brooks Hatlen']),
  ('Jeffrey DeMunn', ['1946 D.A.']),
  ('Larry Brandenburg', ['Skeet']),
  ('Neil Giuntol